In [ ]:
!pip install osmnx

In [110]:

import ee
import geemap
import osmnx as ox
import geopandas as gpd

ee.Authenticate()
ee.Initialize(project='ee-rahmedmoussah12356')


In [111]:
place_name = "Lebel-sur-Quévillon, Quebec, Canada"
gdf = ox.geocode_to_gdf(place_name)
geo_json = gdf.__geo_interface__
region = ee.FeatureCollection(geo_json)
bbox = gdf.total_bounds

print(bbox)

[-77.0719585  49.0030673 -76.9242658  49.0962425]


In [112]:
Apres_date_landsat1 = '2023-06-21'
Apres_date_landsat2 = '2023-08-30'

In [120]:

landsat_image = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterDate(Apres_date_landsat1, Apres_date_landsat2).filterBounds(region.geometry()).sort('CLOUD_COVER').first()
def applyScaleFactors(image):
    optical = image.select('SR_B.*') \
                   .multiply(0.0000275) \
                   .add(-0.2)

    thermal = image.select('ST_B.*') \
                   .multiply(0.00341802) \
                   .add(149.0)

    return (image
            .addBands(optical, overwrite=True)
            .addBands(thermal, overwrite=True))

img = applyScaleFactors(landsat_image)
Vrai_Couleur=img.select(['SR_B4', 'SR_B3', 'SR_B2'])
Fausse_couleur=img.select(['SR_B5', 'SR_B4', 'SR_B3'])



In [121]:
nbr = landsat_image.expression(
    '(NIR - SWIR2) / (NIR + SWIR2)',
    {
        'NIR': landsat_image.select('SR_B5'),
        'SWIR2': landsat_image.select('SR_B7')
    }
).rename('NBR')


In [122]:
nbr_vis = {'min': -1, 'max': 1, 'palette': ['white','yellow','orange','red','darkred']}

In [123]:
Map = geemap.Map()
Map.centerObject(region, 10)
Map.add_basemap("SATELLITE")
# Map.addLayer(nbr.select('NBR'), nbr_vis, 'Landsat NBR')
Map.addLayer(Vrai_Couleur, {'min':0, 'max':0.5}, 'Vrai_couleur')
Map.addLayer(Fausse_couleur, {'min':0, 'max':0.5}, 'Fausse_couleur')

Map.addLayer(region, {'color': 'red'}, 'Lebel-sur-Quévillon')
Map.add_draw_control()
Map

Map(center=[49.03820345229217, -77.00503065138031], controls=(WidgetControl(options=['position', 'transparent_…